In [6]:
import os
import time
import pickle
import itertools
import pandas as pd
from tqdm import tqdm
from iml_group_proj.config import RANDOM_STATE
from iml_group_proj.evaluation import evaluate_many
from iml_group_proj.data.utils import load_libofc_df
from iml_group_proj.data.preprocess import AverageEmbeddingsPerRecord, DataMode
from iml_group_proj.data.embeddings import EmbeddingsDataLoader, EmbeddingsType
from iml_group_proj.trainer import train_model, maybe_load_model

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
DATA_PATH = "../github_data"
CACHE_DIR = "../_cache/"
def experiment_to_file_path(experiment, model_name):
    return os.path.join(CACHE_DIR, f"{experiment[0]}_{experiment[1]}_{model_name}.pk")
        
def get_experiment_result(embeddings_type, data_mode, models):
    print(f"Starting experiment with {embeddings_type} embeddings with {data_mode}")
    data = EmbeddingsDataLoader.load(DATA_PATH, embeddings_type)
    X_train, y_train, X_test, y_test = AverageEmbeddingsPerRecord.prep(data, embeddings_type, data_mode)
    
    trained_models = []
    for model in models:
        trained_model = train_model(
            model,
            X_train,
            y_train,
            {"embeddings_type": embeddings_type, "data": data_mode},
            experiment_to_file_path((embeddings_type, data_mode), model[2])
        )
        
        trained_models.append(trained_model)
    print("Evaluating")
    result_dicts = evaluate_many(trained_models, X_train, y_train, X_test, y_test)

    return result_dicts, trained_models

In [9]:
## Defining each experiment settings
embeddings_types = [
    EmbeddingsType.TFIDF, 
    EmbeddingsType.W2V,
    EmbeddingsType.BERT_XS,
    EmbeddingsType.BERT_S,
    EmbeddingsType.BERT_L
]

data_types = [DataMode.title_only, DataMode.synopsis_only, DataMode.both_title_synopsis]

experiments = list(itertools.product(embeddings_types[-1:], data_types))

models = [
        (MLPClassifier(random_state=1, max_iter=250, hidden_layer_sizes=(600, 600), early_stopping=True), None, 'MLP'),
        (SVC(C=100, kernel="rbf", gamma=0.001, random_state=RANDOM_STATE), None, "SVC"),
        (GaussianNB(), None,'NaiveBayes'),
]

In [12]:
result_dict_list = []
trained_models_list = []
for (embeddings_type, data_type) in tqdm(experiments):
    result_dicts, trained_models = get_experiment_result(embeddings_type, data_type, models)
    print(pd.DataFrame(result_dicts))
    trained_models_list.extend(trained_models)
    result_dict_list.extend(result_dicts)

  0%|                                                                                                                                                                    | 0/3 [00:00<?, ?it/s]

Starting experiment with bert_large embeddings with title
Model found at ../_cache/bert_large_title_MLP.pk, skip training flow...
Model found at ../_cache/bert_large_title_SVC.pk, skip training flow...
Model found at ../_cache/bert_large_title_NaiveBayes.pk, skip training flow...
Evaluating


 33%|███████████████████████████████████████████████████▋                                                                                                       | 1/3 [09:10<18:20, 550.48s/it]

   accuracy  precision    recall  f1_score        name embeddings_type   data  \
0  0.776502   0.785875  0.770462  0.774476         MLP      bert_large  title   
1  0.612667   0.629817  0.612667  0.614628         MLP      bert_large  title   
2  0.791859   0.798189  0.789916  0.792634         SVC      bert_large  title   
3  0.623667   0.633828  0.623667  0.625870         SVC      bert_large  title   
4  0.415615   0.466178  0.406159  0.399275  NaiveBayes      bert_large  title   
5  0.390333   0.435434  0.390333  0.376867  NaiveBayes      bert_large  title   

  best_params  is_train  
0          {}      True  
1          {}     False  
2          {}      True  
3          {}     False  
4          {}      True  
5          {}     False  
Starting experiment with bert_large embeddings with synopsis
Model not found at ../_cache/bert_large_synopsis_MLP.pk, training MLP
Finished training!
Saving the model to ../_cache/bert_large_synopsis_MLP.pk...
Model not found at ../_cache/bert_large_

 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                   | 2/3 [19:50<10:03, 603.05s/it]

   accuracy  precision    recall  f1_score        name embeddings_type  \
0  0.939473   0.941114  0.938654  0.939121         MLP      bert_large   
1  0.733000   0.740670  0.733000  0.733635         MLP      bert_large   
2  0.868565   0.870096  0.867949  0.868584         SVC      bert_large   
3  0.740667   0.745303  0.740667  0.741442         SVC      bert_large   
4  0.550850   0.566894  0.548818  0.549518  NaiveBayes      bert_large   
5  0.532000   0.554923  0.532000  0.533312  NaiveBayes      bert_large   

       data best_params  is_train  
0  synopsis          {}      True  
1  synopsis          {}     False  
2  synopsis          {}      True  
3  synopsis          {}     False  
4  synopsis          {}      True  
5  synopsis          {}     False  
Starting experiment with bert_large embeddings with both
Model not found at ../_cache/bert_large_both_MLP.pk, training MLP
Finished training!
Saving the model to ../_cache/bert_large_both_MLP.pk...
Model not found at ../_cache/be

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [45:09<00:00, 903.21s/it]

   accuracy  precision    recall  f1_score        name embeddings_type  data  \
0  0.884667   0.886185  0.884055  0.883845         MLP      bert_large  both   
1  0.756000   0.757956  0.756000  0.755128         MLP      bert_large  both   
2  0.985270   0.985303  0.985235  0.985253         SVC      bert_large  both   
3  0.751333   0.754292  0.751333  0.751922         SVC      bert_large  both   
4  0.538902   0.564191  0.534759  0.533341  NaiveBayes      bert_large  both   
5  0.506667   0.535751  0.506667  0.503779  NaiveBayes      bert_large  both   

  best_params  is_train  
0          {}      True  
1          {}     False  
2          {}      True  
3          {}     False  
4          {}      True  
5          {}     False  


In [49]:
final_result_df = pd.DataFrame(result_dict_list)
#final_result_df.to_csv("../_output/experiments_result_full.csv")

In [50]:
final_result_df

,accuracy,precision,recall,f1_score,name,embeddings_type,data,best_params,is_train
0,0.776502,0.785875,0.770462,0.774476,MLP,bert_large,title,{},True
1,0.612667,0.629817,0.612667,0.614628,MLP,bert_large,title,{},False
2,0.791859,0.798189,0.789916,0.792634,SVC,bert_large,title,{},True
3,0.623667,0.633828,0.623667,0.625870,SVC,bert_large,title,{},False
4,0.415615,0.466178,0.406159,0.399275,NaiveBayes,bert_large,title,{},True
5,0.390333,0.435434,0.390333,0.376867,NaiveBayes,bert_large,title,{},False
6,0.939473,0.941114,0.938654,0.939121,MLP,bert_large,synopsis,{},True
7,0.733000,0.740670,0.733000,0.733635,MLP,bert_large,synopsis,{},False
8,0.868565,0.870096,0.867949,0.868584,SVC,bert_large,synopsis,{},True
9,0.740667,0.745303,0.740667,0.741442,SVC,bert_large,synopsis,{},False
